# Lab 3 - Training
In this lab we will take our data and train a couple of models using different algorithms and utilize our training cluster

In [ ]:
import numpy
import os
import pandas as pd
import tensorflow as tf

%store -r STUDENT

## Set the project repo 
def ProjectRepo(path):
   ProjectRepo = "/bd-fs-mnt/project_repo"
   return str(ProjectRepo + '/' + path)

## Load the dataset
dataset = pd.read_csv(ProjectRepo('data/' + STUDENT + '_Diabetes/pima-indians-diabetes.csv'), delimiter=",")
dataset.columns = [
    "NumTimesPrg", "PlGlcConc", "BloodP",
    "SkinThick", "TwoHourSerIns", "BMI",
    "DiPedFunc", "Age", "HasDiabetes"]

- Pregnancies
- Glucose — The blood plasma glucose concentration after a 2 hour oral glucose tolerance test.
- BloodPressure — Diastolic blood pressure (mm/HG).
- SkinThickness — Skinfold thickness of the triceps (mm).
- Insulin — 2 hour serum insulin (mu U/ml).
- BMI — Body mass index (kg/m squared)
- DiabetesPedigreeFunction — A function that determines the risk of type 2 diabetes based on family history, the larger the function, the higher the risk of type 2 diabetes.
- Age.
- Outcome — Whether the person is diagnosed with type 2 diabetes (1 = yes, 0 = no).

# Model Development (Part 1)
- Attempting first model with XGB

In [ ]:
# First XGBoost model for Pima Indians dataset
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

# load data
dataset = loadtxt(ProjectRepo('data/' + STUDENT + '_Diabetes/pima-indians-diabetes.csv'), delimiter=",") 

# split data into X and y
X_train = dataset[:,0:8]
y_train = dataset[:,8]

# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)

#
print(model.get_xgb_params())

In [ ]:
%attachments

# Model Development (Part 2)
- Second model using keras with remote training cluster
- Save model and prepare for TensorFlow Serving

## Fill in your initials for the <b>STUDENT</b> variable and your training cluster name

In [ ]:
%%<your training cluster>

import numpy
import os
import pandas as pd
import tensorflow as tf
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

STUDENT = ""

## Set the project repo 
def ProjectRepo(path):
   ProjectRepo = "/bd-fs-mnt/project_repo"
   return str(ProjectRepo + '/' + path)  

## Load the dataset
print("Loading data")
dataset = loadtxt(ProjectRepo('data/' + STUDENT + '_Diabetes/pima-indians-diabetes.csv'), delimiter=",") 
dataset.shape

# Split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

# Define the keras model
print("Building model")
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the keras model on the dataset
print("Training model")
model.fit(X, y, epochs=150, batch_size=10, verbose= 0)

# Evaluate the keras model
_, accuracy = model.evaluate(X, y, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))

# Make class predictions with the model
predictions = model.predict_classes(X)

# Summarize the first 3 cases
for i in range(3):
	print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))

# Save model weights and architecture together
print("Saving model")
model.save(ProjectRepo('models/' + STUDENT + '_Diabetes/db_remote.h5'))

# Evaluate the keras model
_, accuracy = model.evaluate(X, y, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))

# Make class predictions with the model
predictions = model.predict_classes(X)

# Summarize the first 5 cases
for i in range(5):
	print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))
    
# Prepare TF Serving
print("Preparing for TF Serving")
MODEL_VERSION = 1
tf.keras.backend.set_learning_phase(0) 
model = tf.keras.models.load_model(ProjectRepo('models/' + STUDENT + '_Diabetes/db_remote.h5'))
export_path = ProjectRepo('models/' + STUDENT + '_Diabetes/' + str(MODEL_VERSION))
tf.keras.models.save_model(model, export_path)
    
# Summarize model.
model.summary()
print("Done")

## After executing the cell above, a historyl url will be returned. Copy and paste the url into the cell below. You can execute the cell below as many times as you'd like to see the job status in real time. 

In [ ]:
# Fill in the history url from the output of the previous cell
%logs --url <your history url>